In [0]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os

In [0]:
cuda = True if torch.cuda.is_available else False
device = torch.device("cuda:0" if cuda else "cpu")

os.makedirs("./data", exist_ok=True)
os.makedirs("./images", exist_ok=True)

In [0]:
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
])

batch_size = 64
num_workers = 2


dataset = datasets.CIFAR10("./data", download=True,
                        transform=transform
                       )

data_loader = DataLoader(dataset, 
                         batch_size=batch_size,
                         shuffle=True,
                         num_workers=num_workers)


0it [00:00, ?it/s]

170500096it [00:06, 26594661.88it/s]                               


In [0]:
z_dim = 100
ngf = 64
ndf = 64
nc = 3

class Generator(nn.Module):
  
  def block(self, input, size):
    _block = nn.Sequential(
          nn.ConvTranspose2d(input, size, 4, 2, 1, bias=False),
          nn.BatchNorm2d(size),
          nn.ReLU(True)
    )
    return _block
  
  def __init__(self):
    super(Generator, self).__init__()
    self.model = nn.Sequential(
        self.block(z_dim, ngf * 8),
        self.block(ngf * 8, ngf * 4),
        self.block(ngf * 4, ngf * 2),
        self.block(ngf * 2, ngf),
        nn.ConvTranspose2d(ngf, nc, 4,2,1, bias=False),
        nn.Tanh()
    )
  
  def forward(self, x):
    return self.model(x)
  
class Discriminator(nn.Module):
  
  def block(self, input, size):
    _block = nn.Sequential(
           nn.Conv2d(input, size, 4, 2, 1, bias=False),
           nn.BatchNorm2d(size),
           nn.LeakyReLU(0.2, inplace=True)  
    )
    return _block
  
  def __init__(self):
    super(Discriminator, self).__init__()
    self.model = nn.Sequential(
         self.block(nc, ndf),
         self.block(ndf, 2 * ndf),
         self.block(2 * ndf, 4 * ndf),
         self.block(4 * ndf, 8 * ndf),
         nn.Conv2d(8 * ndf, 1, 4, 2, 1, bias=False),
         nn.Sigmoid()
    )
  def forward(self, x, label):
    return self.model(x)

generator = Generator().cuda()
discriminator = Discriminator().cuda()


    

In [0]:
loss_function = nn.BCELoss()

learning_rate = 0.0002
beta1 = 0.5
beta2 = 0.999

d_optimizer = torch.optim.Adam(discriminator.parameters(),
                              lr=learning_rate,
                              betas=(beta1, beta2))
g_optimizer = torch.optim.Adam(generator.parameters(),
                              lr=learning_rate,
                              betas=(beta1, beta2))


z_noise = torch.randn(batch_size, z_dim, 1, 1, device=device)



In [0]:

epochs = 20

for epoch in range(epochs):
  for i, data in enumerate(data_loader, 0):
    
    # update the discriminator
    discriminator.zero_grad()
    real = data[0].cuda()
    
    